## Efficiently Cleaning Text with Pandas

You can read the original article [Here](https://pbpython.com/text-cleaning.html)

Download File if not exists:

In [60]:
%%bash

url=data.iowa.gov/api/views/m3tr-qhgy/rows.csv?accessType=DOWNLOAD

wget $url -O iowa_liquor_sales.csv -nc 

echo $?

1


O arquivo “iowa_liquor_sales.csv” já existe, não será baixado.


In [61]:
%%bash

pip install pandas numpy sidetable

You should consider upgrading via the '/home/junior/repos/github.com/aquilesics/pandas/env/bin/python3.9 -m pip install --upgrade pip' command.


In [62]:
import pandas as pd 
import numpy as np
import sidetable # a package to summarize the data

In [63]:
df = pd.read_csv("iowa_liquor_sales.csv")

In [64]:
df

,Invoice/Item Number,Date,Store Number,Store Name,Address,City,Zip Code,Store Location,County Number,County,...,Item Number,Item Description,Pack,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons)
0,S31296100034,03/16/2016,3762.0,Wine and Spirits Gallery,7690 HICKMAN RD,WINDSOR HEIGHTS,50322.0,NaN,77.0,Polk,...,80457.0,Ryan's Cream Liqueur,12.0,1000.0,7.23,10.85,1.0,10.85,1.0,0.26
1,INV-20599500022,07/15/2019,4320.0,Fareway Stores #470 / Perry,NaN,NaN,NaN,NaN,NaN,NaN,...,10008.0,Scoresby Rare Scotch,6.0,1750.0,10.50,15.75,6.0,94.50,10.5,2.77
2,INV-28403900139,07/01/2020,2643.0,Hy-Vee Wine and Spirits / Waterloo,2126 Kimball Ave,Waterloo,50701.0,NaN,7.0,BLACK HAWK,...,15628.0,Jameson,6.0,1750.0,32.48,48.72,2.0,97.44,3.5,0.92
3,INV-20560900100,07/11/2019,3456.0,Quick Shop / Clear Lake,NaN,NaN,NaN,NaN,NaN,NaN,...,10009.0,Scoresby Rare Scotch,12.0,1000.0,6.74,10.11,2.0,20.22,2.0,0.52
4,INV-20784900043,07/23/2019,5151.0,IDA Liquor,"500, Hwy 175",Ida Grove,51445.0,NaN,47.0,IDA,...,100107.0,Captain Morgan OSR Glass w/50ml Apple Smash & ...,6.0,1850.0,18.00,27.00,6.0,162.00,11.1,2.93
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
741913,INV-26179600014,03/30/2020,5255.0,Whole Foods Market,4100 U{,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
741914,"""error"" : true",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
741915,"""message"" : ""Internal error""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
741916,"""status"" : 500",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
